# Infer GOLGI - part 6️⃣

--------------

## OBJECTIVE:  ✅ Infer sub-cellular component GOLGI COMPLEX  in order to understand interactome 



Dependencies:
The GOLGI  inference rely on the CYTOSOL, which is SOMA&~NUCLEI.  



## IMPORTS

In [1]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict

import numpy as np
import scipy

# TODO:  prune the imports.. this is the big set for almost all organelles
# # function for core algorithm
from scipy import ndimage as ndi
import aicssegmentation
from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.utils import topology_preserving_thinning, size_filter
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.utils import hole_filling
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice
from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation, remove_small_holes   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc_2d
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc_2d.utils.file_io import (read_input_image, 
                                                                    list_image_files, 
                                                                    export_ome_tiff, 
                                                                    etree_to_dict, 
                                                                    save_parameters, 
                                                                    load_parameters, 
                                                                    export_ndarray)
from infer_subc_2d.utils.img import *
from infer_subc_2d.organelles.nuclei import infer_NUCLEI
from infer_subc_2d.organelles.soma import infer_SOMA
from infer_subc_2d.organelles.cytosol import infer_CYTOSOL
from infer_subc_2d.constants import TEST_IMG_N

%load_ext autoreload
%autoreload 2

test_img_n = TEST_IMG_N

# IMAGE PROCESSING Objective 6:  infer GOLGI COMPLEX
> Back to  [OUTLINE: Objective #5](00_pipeline_setup.ipynb#summary-of-objectives)

## summary of steps (Workflow #1 & #2)

INPUT
- channel  4
- CY mask

PRE-PROCESSING
-  smoothe / remove noise

CORE-PROCESSING
-  segment objects

- POST-PROCESSING
  - filter objects

OUTPUT
- object GOLGI 



------------------------
# LOAD RAW IMAGE DATA
Identify path to _raw_ image data and load our example image


In [2]:
# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(data_path,im_type)
test_img_name = img_file_list[test_img_n]

bioim_image = read_input_image(test_img_name)
img_data = bioim_image.image
raw_meta_data = bioim_image.raw_meta
ome_types = []
meta_dict = bioim_image.meta

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


## Get default parameters, including  "optimal" Z

takes ~ 4 seconds to calculate

In [3]:
load_Z_from_params = False


if load_Z_from_params:

    default_params = load_parameters( test_img_name.split("/")[-1], data_root_path / "intermediate" )

    ch_to_agg = default_params["ch_to_agg"]
    nuc_ch = default_params['nuc_ch']
    optimal_Z = default_params["optimal_Z"] #find_optimal_Z(img_data, nuc_ch, ch_to_agg) 
else:
    ch_to_agg = (1,2,3,4,5,6)
    nuc_ch = 0
    optimal_Z = find_optimal_Z(img_data, nuc_ch, ch_to_agg) 

    default_params = defaultdict(str, **{
        #"intensity_norm_param" : [0.5, 15]
        "intensity_norm_param" : [0],
        "gaussian_smoothing_sigma" : 1.34,
        "gaussian_smoothing_truncate_range" : 3.0,
        "dot_2d_sigma" : 2,
        "dot_2d_sigma_extra" : 1,
        "dot_2d_cutoff" : 0.025,
        "min_area" : 10,
        "low_level_min_size" :  100,
        "median_filter_size" : 4,
        "ch_to_agg" : (1,2,3,4,5,6), # exclude residual
        "nuc_ch" : 0,
        "optimal_Z": optimal_Z,
    })
    save_parameters(default_params, test_img_name.split("/")[-1], data_root_path / "intermediate" )
# make sure we have removed Z
if len(scale)>2:
    scale = scale[1:]


img_2D = img_data[:,[optimal_Z],:,:].copy()


## get the inferred nuclei object

(takes < 1 sec)

In [4]:


raw_nuclei = img_2D[0].copy()
NU_object, NU_label, out_p =  infer_NUCLEI(raw_nuclei.copy(), default_params) 


intensity normalization: min-max normalization with NO absoluteintensity upper bound


## get the inferred soma object

(takes < 1 sec)

In [5]:
raw_soma = (4. * img_2D[1].copy() + 
                               1. * img_2D[5].copy() + 
                               1. * img_2D[7].copy() )

SO_object, SO_label, out_p =  infer_SOMA(raw_soma.copy(), NU_label, default_params) 
CY_object =  infer_CYTOSOL(SO_object, NU_object) 


intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound


# WORKFLOW #1 

Generally following the Allen Cell Segmenter procedure, but doing more aggressive contrast scaling than their prescribed contrast scaling.

> As per the Allen Cell segmenter sialyltransferase 1 (ST6GAL1) a potential Golgi segmenter. e.g. [Allen Cell](https://www.allencell.org/cell-observations/category/golgi-apparatus).    using [seg_st6gal1.py]("../../../../aics-segmentation/aicssegmentation/structure_wrapper/seg_st6gal1.py") and [playground_st6gal1.ipynb]("../../../../aics-segmentation/lookup_table_demo/playground_st6gal1.ipynb")



## summary of steps

INPUT
- ch 4
- CY mask

PRE-PROCESSING
-   non-local noise reduction
  - size:4, distance:2, cut-off:0.1

CORE-PROCESSING
- adaptive Otsu
    - diameter: (4,100)
  - two classes
    - threshold smoothing scale: 1.34
    - threshold correction factor: .75
    - threshold bounds: (0.14, 1)
    - adaptive window: 20 pixels

  - adaptive Sauvola
    - threshold smoothing scale: 0
    - threshold correction factor: .6
    - threshold bounds: (0. ,1.0)
    - adaptive window: 20 pixels

- POST-PROCESSING
  - N/A

OUTPUT
- object GOLGI 


>#### ASIDE: Perform topology-preserving thinning 
>There are two parameters:
>* `thin_dist_preserve`: Half of the minimum width you want to keep from being thinned. For example, when the object width is smaller than 4, you don't want to make this part even thinning (may break the thin object and alter the topology), you can set `thin_dist_preserve` as `2`.
>* `thin_dist`: the amount to thin (has to be an positive integer). The number of pixels to be removed from outter boundary towards center


### INPUT

In [6]:

###################
# INPUT
###################
raw_golgi   = img_2D[3].copy()



### PRE-PROCESSING

In [7]:
###################
# PRE_PROCESSING
###################


intensity_norm_param = [0.1, 30.]  # CHECK THIS
# Linear-ish smoothing
golgi = intensity_normalization( masked_golgi ,  scaling_param=intensity_norm_param)

med_filter_size =3  

gaussian_smoothing_sigma = 1.34
gaussian_smoothing_truncate_range = 3.0

struct_img = median_filter(golgi,    size=med_filter_size  )

structure_img_smooth = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                                        truncate_range=gaussian_smoothing_truncate_range,
                                                                                                                    )


struct_img = structure_img_smooth




### CORE PROCESSING

In [9]:
###################
# CORE_PROCESSING
###################
################################
## PARAMETERS for this step ##
cell_wise_min_area = 1200
dot_2d_sigma = 1.6
dot_2d_cutoff = 0.02
minArea = 10
thin_dist = 1
thin_dist_preserve = 1.6



###################
# CORE_PROCESSING
###################

bw, object_for_debug = MO(structure_img_smooth, 
                                                global_thresh_method='tri', 
                                                object_minArea=cell_wise_min_area, 
                                                return_object=True)

thin_dist_preserve=1.6
thin_dist=1
bw_thin = topology_preserving_thinning(bw>0, thin_dist_preserve, thin_dist)


################################
## PARAMETERS for this step ##
#s3_param = [[1.6, 0.02]]
s3_param = [(dot_2d_sigma,dot_2d_cutoff)]
################################

bw_extra = dot_3d_wrapper(structure_img_smooth, s3_param)

bw_combine = np.logical_or(bw_extra>0, bw_thin)


### POST-PROCESSING

In [12]:

###################
# POST_PROCESSING
###################

small_object_max = 4

GO_object = size_filter_2D(bw, 
                                                min_size= small_object_max**2, 
                                                connectivity=1)


#### Visualize with `napari`
Visualize the first-pass segmentation and labeling with `napari`.

In [11]:

viewer = napari.view_image(
    bw_combine,
    scale=scale
)

viewer.scale_bar.visible = True
viewer.add_image(
    structure_img_smooth,
    scale=scale
)


<Image layer 'GO_object' at 0x149c7cf10>

In [13]:

viewer.add_image(
    GO_object,
    scale=scale
)

<Image layer 'GO_object [1]' at 0x17f586e50>

### DEFINE `infer_GOLGI` function

In [22]:
##########################
#  infer_GOLGI
##########################
def _infer_GOLGI(struct_img, CY_object,  in_params) -> tuple:
    """
    Procedure to infer GOLGI COMPLEX  from linearly unmixed input.

    Parameters:
    ------------
    struct_img: np.ndarray
        a 2d image containing the GOLGI signal

    CY_object: np.ndarray boolean
        a 2d image containing the NU labels

    in_params: dict
        holds the needed parameters

    Returns:
    -------------
    tuple of:
        object
            mask defined boundaries of MITOCHONDRIA
        parameters: dict
            updated parameters in case any needed were missing
    """
    out_p= in_params.copy()
    
    struct_img = apply_mask(struct_img,CY_object)

    ###################
    # PRE_PROCESSING
    ###################                         
    intensity_norm_param = [0.1, 30.]  # CHECK THIS

    struct_img = intensity_normalization(struct_img, scaling_param=intensity_norm_param)
    out_p["intensity_norm_param"] = intensity_norm_param

   # make a copy for post-post processing
    scaled_signal = struct_img.copy()

    med_filter_size = 3   
    struct_img = median_filter(struct_img,    size=med_filter_size  )
    out_p["median_filter_size"] = med_filter_size 

    gaussian_smoothing_sigma = 1.34
    gaussian_smoothing_truncate_range = 3.0
    struct_img = image_smoothing_gaussian_slice_by_slice(   struct_img,
                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                        truncate_range = gaussian_smoothing_truncate_range
                                                                                                    )
    out_p["gaussian_smoothing_sigma"] = gaussian_smoothing_sigma 
    out_p["gaussian_smoothing_truncate_range"] = gaussian_smoothing_truncate_range



   ###################
    # CORE_PROCESSING
    ###################
    cell_wise_min_area = 1200
    bw, object_for_debug = MO(struct_img, 
                                                global_thresh_method='tri', 
                                                object_minArea=cell_wise_min_area, 
                                                return_object=True)
    out_p["cell_wise_min_area"] = cell_wise_min_area 

    thin_dist_preserve=1.6
    thin_dist=1
    bw_thin = topology_preserving_thinning(bw>0, thin_dist_preserve, thin_dist)
    out_p["thin_dist_preserve"] = thin_dist_preserve 
    out_p["thin_dist"] = thin_dist 

    dot_2d_sigma = 1.6
    dot_2d_cutoff = 0.02
    s3_param = [(dot_2d_sigma,dot_2d_cutoff)]

    bw_extra = dot_3d_wrapper(struct_img, s3_param)
    out_p["dot_2d_sigma"] = dot_2d_sigma 
    out_p["dot_2d_cutoff"] = dot_2d_cutoff 
    out_p["s3_param"] = s3_param 

    bw = np.logical_or(bw_extra>0, bw_thin)


    ###################
    # POST_PROCESSING
    ###################

    small_object_max = 4
    struct_obj = size_filter_2D(bw, 
                                                min_size= small_object_max**2, 
                                                connectivity=1)
    out_p['small_object_max'] = small_object_max


    retval = (struct_obj,  out_p)
    return retval


# TEST  `infer_GOLGI` function

In [23]:

###################
# INPUT
###################
raw_golgi    = img_2D[3].copy()

GL_object, out_p =  _infer_GOLGI(raw_golgi, CY_object, default_params) 

In [30]:

viewer.add_image(
    GL_object,
    scale=scale
)
viewer.add_labels(
    label(GL_object),
    scale=scale
)

<Labels layer 'Labels [5]' at 0x1803eb400>

In [29]:
from infer_subc_2d.organelles.golgi import infer_GOLGI


img_2D = img_data[:,[optimal_Z],:,:].copy()
raw_golgi    = img_2D[3].copy()

GL_object, out_p =  infer_GOLGI(raw_golgi, CY_object, default_params) 


---------------------

 🚧 WIP 🚧 (🚨🚨🚨🚨 )

 
# WORKFLOW #2 (WIP)
as per 6/22 CellProfiler pipeline from MCZ
 
## summary of steps

INPUT
- ch 4
- CY mask

PRE-PROCESSING
-   non-local noise reduction
  - size:4, distance:2, cut-off:0.08

CORE-PROCESSING
- adaptive Otsu
    - diameter: (4,100)
  - two classes
    - threshold smoothing scale: 1.34
    - threshold correction factor: .75
    - threshold bounds: (0.14, 1)
    - adaptive window: 20 pixels

- POST-PROCESSING
  - N/A

OUTPUT
- object GOLGI 



### INPUT

In [ ]:

###################
# INPUT
###################
struct_img_raw = img_data[3,:,:,:].copy()

# DEFAULT PARAMETERS:
intensity_norm_param = [3.5, 15] # from Allen Cell Segmenter LAMP1  workflow
scaling_param = [0]
gaussian_smoothing_sigma = 1.
gaussian_smoothing_truncate_range = 3.0
dot_2d_sigma = 2
dot_2d_sigma_extra = 1
dot_2d_cutoff = 0.025
min_area = 10
low_level_min_size =  100

med_filter_size =2  

gaussian_smoothing_sigma = 10
gaussian_smoothing_truncate_range = 3.0

aicssegmentation.core.pre_processing_utils.suggest_normalization_param(struct_img_raw) #  [0., 23]


### PRE-PROCESSING

In [ ]:
###################
# PRE_PROCESSING
###################

intensity_norm_param = [0] # 

# Linear-ish smoothing
raw_mito = intensity_normalization( struct_img_raw ,  scaling_param=intensity_norm_param)

med_filter_size =3  

gaussian_smoothing_sigma = 1.3
gaussian_smoothing_truncate_range = 3.0

struct_img = median_filter(raw_mito,    size=med_filter_size  )

structure_img_smooth = image_smoothing_gaussian_3d(   struct_img,
                                                                                                                        sigma=gaussian_smoothing_sigma,
                                                                                                                        truncate_range=gaussian_smoothing_truncate_range,
                                                                                                                    )


# log_img, d = log_transform( structure_img_smooth ) 
# struct_img = intensity_normalization(  log_img  ,  scaling_param=[0] )  

struct_img = structure_img_smooth

### CORE PROCESSING

In [ ]:
###################
# CORE_PROCESSING
###################
def _enhance_speckles(image, radius, volumetric=True):
    radius = radius / 2
    if volumetric:
        selem = ball(radius)
    else:
        selem = disk(radius)     

    # if radius >10:
    #         minimum = scipy.ndimage.minimum_filter(image, footprint=selem)
    #         maximum = scipy.ndimage.maximum_filter(minimum, footprint=selem)
    #         result = data - maximum
    # else:
    result =  white_tophat(image)

    return result

def enhance_neurites(image, radius, volumetric = True):
    if volumetric:
        selem = ball(radius)
    else:
        selem = disk(radius)     
    white = white_tophat(image,selem)
    black = black_tophat(image, selem)
    result = image + white - black
    result[result > 1] = 1
    result[result < 0] = 0

    return result

    



In [ ]:
# enhance spreckles - 40
big_struct_rad = 40
big_img = _enhance_speckles(struct_img.copy(),big_struct_rad, True)



In [ ]:

# enhance spreckles - 10
sm_struct_rad = 20
sm_img = _enhance_speckles(struct_img.copy(),sm_struct_rad, True)


#adaptive_otsu(big_struct) # three class - middle foreground
# adaptive window- 20
#size: 10,100
# threshold smooth 1.34
# threshold correction 1
# threshold (0.1497,1)
#fill holes
# "adatpive" thresholds are NOT currently working... will use      
# "Masked Object Thresholding" - 3D
low_level_min_size = 10
bw_big, _bw_low_level = MO(big_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= 0.5, 
                                                return_object=True,
                                                dilate=True)
 # or this?                                               
#    struct_obj = struct_img > filters.threshold_otsu(struct_img)
#     threshold_value_log = threshold_otsu_log(struct_img)

#     threshold_factor = 0.9 #from cellProfiler
#     thresh_min = .1
#     thresh_max = 1.
#     threshold = min( max(threshold_value_log*threshold_factor, thresh_min), thresh_max)
#     struct_obj = struct_img > threshold


# enhance speckles
#   adaptive_sauvola(sm_struct) 
# adaptive window- 10
#size: 2,10
# threshold smooth 1.34
# threshold correction 1
# threshold (0.05,1)
#fill holes
# "adatpive" thresholds are NOT currently working... will use      
# "Masked Object Thresholding" - 3D
low_level_min_size = 2
bw_sm, _bw_low_level = MO(sm_img, 
                                                global_thresh_method='ave', 
                                                object_minArea=low_level_min_size, 
                                                extra_criteria=True,
                                                local_adjust= 0.5, 
                                                return_object=True,
                                                dilate=True)
 # or this?                                               
#    struct_obj = struct_img > filters.threshold_otsu(struct_img)
#     threshold_value_log = threshold_otsu_log(struct_img)
#     threshold_factor = 0.9 #from cellProfiler
#     thresh_min = .1
#     thresh_max = 1.
#     threshold = min( max(threshold_value_log*threshold_factor, thresh_min), thresh_max)
#     struct_obj = struct_img > threshold



### POST-PROCESSING

In [ ]:
###################
# POST_PROCESSING
###################


# 3D
# cleaned_img = remove_small_objects(removed_holes>0, 
#                                                             min_size=minArea, 
#                                                             connectivity=1, 
#                                                             in_place=False)
small_object_max = 10
cleaned_img_big = size_filter(bw_big, # wrapper to remove_small_objects which can do slice by slice
                                                         min_size= small_object_max**2,
                                                         method = "slice_by_slice",
                                                         connectivity=1)


#                                                             in_place=False)
small_object_max = 2
cleaned_img_sm = size_filter(bw_sm, # wrapper to remove_small_objects which can do slice by slice
                                                         min_size= small_object_max**2,
                                                         method = "slice_by_slice",
                                                         connectivity=1)



cleaned_img = np.logical_or(cleaned_img_big, cleaned_img_sm)



#### Visualize with `napari`
Visualize the first-pass segmentation and labeling with `napari`.

In [ ]:

if viewer is None:
    viewer = napari.view_image(
        cleaned_img,
        scale=scale
    )
else: 
    viewer.add_image(
        cleaned_img,
        scale=scale
    )

viewer.scale_bar.visible = True
viewer.add_image(
    cleaned_img_big,
    scale=scale
)

viewer.add_image(
    big_img,
    scale=scale
)
viewer.add_image(
    cleaned_img_big,
    scale=scale
)
viewer.add_image(
    sm_img,
    scale=scale
)

In [ ]:
# save updated parameters for ongoing testing
save_parameters(default_params, test_img_name.split("/")[-1], data_root_path / "intermediate" )

'/Users/ahenrie/Projects/Imaging/data/intermediate/ZSTACK_PBTOhNGN2hiPSCs_BR3_N14_Unmixed.czi_params.pkl'